In [68]:
import pdfplumber
import re
import json
import os
import PyPDF2


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict

In [69]:
file_path = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"
load_dotenv()

True

In [70]:
# una semplice estrazione dei capitoli dal pdf
def extract_chapters_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"

    # Suddividiamo il testo nei capitoli basandoci su "CAPITOLO X"
    chapters = re.split(r'(Capitolo \d+)', text)
    
    structured_data = {}
    for i in range(1, len(chapters), 2):
        chapter_title = chapters[i].strip()
        chapter_content = chapters[i + 1].strip()
        structured_data[chapter_title] = chapter_content
    
    return structured_data

# Leggiamo il PDF e estraiamo i capitoli
pdf_chapters = extract_chapters_from_pdf(file_path)

# Stampiamo i titoli dei capitoli
print("📖 Capitoli trovati:")
for chapter in pdf_chapters.keys():
    print(f"- {chapter}")

📖 Capitoli trovati:
- Capitolo 1
- Capitolo 2
- Capitolo 3
- Capitolo 4
- Capitolo 5


In [71]:
pdf_chapters["Capitolo 3"]

": Tecniche di Preparazione\nNel vasto universo della cucina spaziale, la preparazione è la chiave per ogni grande piatto. In questo\ncapitolo, esploreremo le tecniche che trasformano gli ingredienti da semplici materie prime a protagonisti\ndi un'esperienza culinaria stellare. Ogni passaggio è fondamentale per creare la base perfetta per la\ncottura. Preparatevi a scoprire come l'arte della preparazione, nel cosmo, richieda precisione e ingegno,\nma soprattutto visione.\nMarinatura\nLa marinatura spaziale è un'arte che sfrutta il tempo e lo spazio, dove ogni condimento si mescola in\nperfetta armonia con gli ingredienti. In assenza di gravità, le molecole si distribuiscono in modo unico,\namplificando sapori e aromi. Imparare a marinare nel vuoto cosmico è fondamentale per ottenere piatti\nche sfidano ogni legge della fisica culinaria, rivelando gusti straordinari in ogni morso.\nMarinatura a Infusione Gravitazionale\nCome funziona: il cibo è collocato in una camera a gravità alternat

In [73]:
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")


def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )


In [112]:
def estrai_tecniche_e_macro(llm: AzureChatOpenAI, testo: str) -> tuple[dict, List[str]]:
    """
    Estrae le macrotecniche con descrizioni e le tecniche specifiche dal testo.
    
    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico
        testo (str): Il testo da analizzare
        
    Returns:
        tuple[dict, List[str]]: 
            - Dizionario delle macrotecniche con le loro descrizioni
            - Lista delle tecniche specifiche
    """
    # Prima estrazione: macrotecniche con descrizioni
    prompt_macro = """
    Analizza il seguente testo e identifica le macrotecniche culinarie.
    Restituisci un oggetto JSON con il nome della macrotecnica come chiave e la sua descrizione come valore.
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.
    
    Testo:
    {}
    """.format(testo)
    
    response_macro = llm.invoke(prompt_macro)
    content_macro = response_macro.content.strip()
    
    # Pulizia del JSON delle macrotecniche
    if content_macro.startswith('```') and content_macro.endswith('```'):
        content_macro = '\n'.join(content_macro.split('\n')[1:-1])
    if content_macro.startswith('json'):
        content_macro = content_macro[4:].strip()
    
    try:
        macrotecniche = json.loads(content_macro)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle macrotecniche:", content_macro)
        macrotecniche = {}
    
    # Seconda estrazione: tecniche specifiche
    prompt_tecniche = """
    Analizza il seguente testo e elenca SOLO le tecniche specifiche di preparazione (non le macrotecniche).
    Elenca solo i nomi delle tecniche, una per riga, senza punteggiatura o numerazione.
    Non aggiungere spiegazioni o altro testo.
    
    Testo:
    {}
    """.format(testo)
    
    response_tecniche = llm.invoke(prompt_tecniche)
    
    # Elaborazione delle tecniche specifiche
    tecniche = [
        riga.strip() 
        for riga in response_tecniche.content.split('\n') 
        if riga.strip() and not riga.startswith('-') and not riga[0].isdigit()
    ]
    
    # Rimuovi duplicati mantenendo l'ordine
    tecniche = list(dict.fromkeys(tecniche))
    
    return macrotecniche, tecniche

In [ ]:
def estrai_dettagli_tecniche_per_grafo(llm: AzureChatOpenAI, testo: str, tecniche: List[str]) -> dict:
    """
    Estrae informazioni strutturate per ogni tecnica dal testo, ottimizzate per GraphRAG.
    
    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico
        testo (str): Il testo originale da analizzare
        tecniche (List[str]): Lista delle tecniche trovate
        
    Returns:
        dict: Dizionario strutturato per costruzione del grafo
    """
    prompt_template = """
    Analizza la tecnica culinaria "{tecnica}" nel seguente testo e restituisci le informazioni in questo formato JSON strutturato:
    {{
        "tecnica": "nome della tecnica",
        "come_funziona": "breve descrizione del funzionamento della tecnica",
        "vantaggi": "vantaggi della tecnica",
        "svantaggi": "svantaggi della tecnica"
    }}
    Restituisci SOLO il JSON, senza formattazione markdown o altri decoratori.
    
    Testo da analizzare:
    {testo}
    """
    
    dettagli_grafo = {}
    
    for tecnica in tecniche:
        prompt = prompt_template.format(tecnica=tecnica, testo=testo)
        response = llm.invoke(prompt)
        
        try:
            # Assumendo che la risposta sia in formato JSON valido
            import json
            dettagli_grafo[tecnica] = json.loads(response.content)
        except json.JSONDecodeError:
            # Fallback se il JSON non è valido
            dettagli_grafo[tecnica] = {"error": "Formato non valido", "raw_content": response.content}
    
    return dettagli_grafo

In [113]:
# Utilizzo:
llm = crea_llm(api_key, api_base)
testo_input = pdf_chapters["Capitolo 3"]

In [114]:
macrotecniche, tecniche = estrai_tecniche_e_macro(llm, testo_input)

# Per visualizzare i risultati:
print("\nMacrotecniche:")
for macro, descrizione in macrotecniche.items():
    print(f"\n{macro}:")
    print(descrizione)

print("\nTecniche specifiche:")
for tecnica in tecniche:
    print(f"- {tecnica}")


Macrotecniche:

Marinatura:
La marinatura spaziale è un'arte che sfrutta il tempo e lo spazio, dove ogni condimento si mescola in perfetta armonia con gli ingredienti. In assenza di gravità, le molecole si distribuiscono in modo unico, amplificando sapori e aromi. Imparare a marinare nel vuoto cosmico è fondamentale per ottenere piatti che sfidano ogni legge della fisica culinaria, rivelando gusti straordinari in ogni morso.

Affumicatura:
L'affumicatura è una tecnica che va ben oltre la semplice espressione di fumo. Qui, lʼarte di infondere sapore sfrutta il vuoto cosmico per distribuire il fumo in modo uniforme e affascinante, regalando ai piatti un aroma profondo che si espande in modo inaspettato. La gestione della temperatura e del flusso di fumo è fondamentale per ottenere un affumicato perfetto, che aggiunga una dimensione nuova e sorprendente a ogni ingrediente, senza alterarne la qualità.

Fermentazione:
La fermentazione galattica è una danza di batteri e microrganismi che tr

In [115]:
dettagli_per_grafo = estrai_dettagli_tecniche_per_grafo(llm, testo_input, tecniche)

In [125]:
dettagli_per_grafo

{'Marinatura a Infusione Gravitazionale': {'tecnica': 'Marinatura a Infusione Gravitazionale',
  'come_funziona': 'Il cibo è collocato in una camera a gravità alternata, che intensifica lʼassorbimento della marinata nei tessuti alimentari.',
  'vantaggi': 'Il sapore penetra in profondità per una marinatura uniforme e intensa.',
  'svantaggi': 'Necessita di un “Grav-Infuser” di alta precisione, con un dispendio energetico significativo.'},
 'Marinatura Temporale Sincronizzata': {'tecnica': 'Marinatura Temporale Sincronizzata',
  'come_funziona': 'sfrutta una bolla temporale sincronizzata per accelerare il processo di marinatura, comprimendo ore in minuti.',
  'vantaggi': 'marinatura completa in un tempo minimo, senza compromessi sulla qualità.',
  'svantaggi': 'rischi di interferenze temporali minori; vietata per uso domestico senza supervisione.'},
 'Marinatura Psionica': {'tecnica': 'Marinatura Psionica',
  'come_funziona': "Uno chef telepate concentra la propria energia mentale per m

In [120]:
# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

In [121]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [ ]:
# Funzione per pulire il database
def pulisci_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")

# Pulizia del database
with driver.session() as session:
    session.execute_write(pulisci_database)

print("Database pulito con successo!")

C:\Users\lparigi\AppData\Local\Temp\ipykernel_2084\3421409616.py:7: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(pulisci_database)


Database pulito con successo!


In [ ]:
def aggiungi_macrotecniche(tx, nome, descrizione):
    query = """
    MERGE (m:Macrotecnica {nome: $nome})
    SET m.descrizione = $descrizione
    """
    tx.run(query, nome=nome, descrizione=descrizione)

# Inserimento dati nel database
with driver.session() as session:
    for macro, descrizione in macrotecniche.items():
        session.execute_write(aggiungi_macrotecniche, macro, descrizione)

print("Macrotecniche aggiunte con successo!")

C:\Users\lparigi\AppData\Local\Temp\ipykernel_2084\392791697.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(aggiungi_macrotecniche, macro, descrizione)


Macrotecniche aggiunte con successo!


In [126]:
# Funzione per aggiungere le tecniche
def aggiungi_tecniche(tx, nome, come_funziona, vantaggi, svantaggi):
    query = """
    MERGE (t:Tecnica {nome: $nome})
    SET t.come_funziona = $come_funziona,
        t.vantaggi = $vantaggi,
        t.svantaggi = $svantaggi
    """
    tx.run(query, nome=nome, come_funziona=come_funziona, vantaggi=vantaggi, svantaggi=svantaggi)

# Inserire le tecniche nel database
with driver.session() as session:
    for nome, dettagli in dettagli_per_grafo.items():
        session.execute_write(
            aggiungi_tecniche, nome, dettagli["come_funziona"], dettagli["vantaggi"], dettagli["svantaggi"]
        )

print("Tecniche aggiunte con successo!")

Tecniche aggiunte con successo!


In [118]:
# Poi crea il knowledge graph
graph_creator = KnowledgeGraphCreator(
    uri=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Pulisci il database (opzionale)
graph_creator.clear_database()

In [119]:
# Crea il grafo
graph_creator.create_knowledge_graph(dettagli_per_grafo)

AttributeError: 'str' object has no attribute 'nome'

In [ ]:
# Crea le relazioni tra tecniche
graph_creator.create_technique_relationships()

# Esempio di query: trova tecniche correlate
related = graph_creator.get_related_techniques("Marinatura")
print("Tecniche correlate:", related)

# Ottieni un riepilogo di una tecnica
summary = graph_creator.get_technique_summary("Marinatura")
print("Riepilogo tecnica:", summary)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: CONDIVIDE_VANTAGGI)} {position: line: 3, column: 40, offset: 88} for query: "\n                MATCH (t:Tecnica {nome: $nome})\n                OPTIONAL MATCH (t)-[rv:CONDIVIDE_VANTAGGI]->(tv:Tecnica)\n                OPTIONAL MATCH (t)-[rs:CONDIVIDE_SVANTAGGI]->(ts:Tecnica)\n                RETURN \n                    collect(DISTINCT {nome: tv.nome, tipo: 'vantaggi', count: rv.count}) as per_vantaggi,\n                    collect(DISTINCT {nome: ts.nome, tipo: 'svantaggi', count: rs.count}) as per_svantaggi\n            "
Received 

Tecniche correlate: {'relazioni_per_vantaggi': [], 'relazioni_per_svantaggi': []}


AttributeError: 'NoneType' object has no attribute 'data'